In [1]:
from typing import List
import math
import pandas as pd
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
import vertexai
PROJECT_ID = "example"

In [2]:
def text_to_vector(texts: list = None,dimensionality:int = 256) -> list[list[float]]:
    """Embeds texts with a pre-trained, foundational model.
    Inputs:
    texts:
        list of texts
    dimentionality :
        The dimensionality of the output embeddings.

    Returns:
        A list of lists containing the embedding vectors for each input text
    """
    
    # The task type for embedding. Check the available tasks in the model's documentation.
    task = "SEMANTIC_SIMILARITY" 

    model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")
    inputs = [TextEmbeddingInput(text, task) for text in texts]
    kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
    embeddings = model.get_embeddings(inputs, **kwargs)

    return [embedding.values for embedding in embeddings]


In [3]:
text = """私は東京科学大学の学生です。
現在は学部3年生で情報工学系に所属しています。
高校まではアメリカに住んでいました。
まだ研究室には所属していませんが、自然言語処理に興味があります。
東京科学大学は自然言語処理に関連する研究室が多くあるので、最近は研究室訪問を積極的に行なっています。
自然言語処理に関する授業は大学院から始まります。
学部4年では大学院科目を10単位まで履修できるので、来年に受講したいと考えています。
学部の卒業後は大学院への進学を考えており、引き続き東京科学大学へ進学したいです。
"""

lines = text.splitlines() #改行区切りのリスト

lines.append(text) #末尾に元文章を追加

In [4]:
vertexai.init(project=PROJECT_ID, location="us-central1")
vectors = text_to_vector(lines) #ベクトル化

In [5]:
def cosine_similarity(vec1: List[float], vec2: List[float]) -> float:
    """
    リスト形式のベクトル同士でコサイン類似度を計算する関数。
    
    Parameters:
        vec1 (List[float]): 1つ目のベクトル
        vec2 (List[float]): 2つ目のベクトル
    
    Returns:
        float: コサイン類似度
    """
    # 内積の計算
    dot_product = sum(x * y for x, y in zip(vec1, vec2))
    
    # 各ベクトルの大きさを計算
    norm1 = math.sqrt(sum(x**2 for x in vec1))
    norm2 = math.sqrt(sum(x**2 for x in vec2))
    
    # ゼロ除算を防ぐ
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    # コサイン類似度を計算
    return dot_product / (norm1 * norm2)


# ベクトル辞書を作成
# 
titles = [
    line if len(line) <= 10 else line[:9] + "..." for line in lines
]
titles[-1] = "文章"

vector_dict = {title: vector for title, vector in zip(titles, vectors)}

# コサイン類似度を計算して表を作成
vector_names = list(vector_dict.keys())
cosine_similarity_table = pd.DataFrame(index=vector_names, columns=vector_names)

for vec1_name in vector_names:
    for vec2_name in vector_names:
        cosine_similarity_table.loc[vec1_name, vec2_name] = cosine_similarity(
            vector_dict[vec1_name], vector_dict[vec2_name]
        )

# 表を出力
cosine_similarity_table

,私は東京科学大学の...,現在は学部3年生で...,高校まではアメリカ...,まだ研究室には所属...,東京科学大学は自然...,自然言語処理に関す...,学部4年では大学院...,学部の卒業後は大学...,文章
私は東京科学大学の...,1.0,0.777258,0.736904,0.738033,0.735958,0.669185,0.666497,0.805173,0.838432
現在は学部3年生で...,0.777258,1.0,0.679331,0.761122,0.679594,0.696927,0.683391,0.734237,0.846423
高校まではアメリカ...,0.736904,0.679331,1.0,0.677995,0.636139,0.661294,0.671223,0.70746,0.792628
まだ研究室には所属...,0.738033,0.761122,0.677995,1.0,0.781103,0.80174,0.615446,0.663863,0.834153
東京科学大学は自然...,0.735958,0.679594,0.636139,0.781103,1.0,0.763356,0.675837,0.741359,0.869333
自然言語処理に関す...,0.669185,0.696927,0.661294,0.80174,0.763356,1.0,0.705527,0.698927,0.82612
学部4年では大学院...,0.666497,0.683391,0.671223,0.615446,0.675837,0.705527,1.0,0.781323,0.772911
学部の卒業後は大学...,0.805173,0.734237,0.70746,0.663863,0.741359,0.698927,0.781323,1.0,0.863952
文章,0.838432,0.846423,0.792628,0.834153,0.869333,0.82612,0.772911,0.863952,1.0
